# 1.0 Introduction 

# 2.0 Authenticating with Twitter's API 

In [1]:
!pip install Twython

     |████████████████████████████████| 151 kB 9.8 MB/s eta 0:00:01


In [2]:
from twython import Twython, TwythonError

## 2.1 Enter app info and get auth URL 

In order to authenticate with Twitter, we'll provide the app details and ask for a one-time authorization URL to authenticate your user with this app.

Copy and paste the API key and secret from your Twitter app into a file named keys.txt. The first line is the API_KEY and the second line of the file is API_SECRET_KEY. For example, a template for the keys.txt:

In [ ]:
# %load keys.txt
GRvkTm3qkMOM8YjyNp3S6xLO4
YmHFrSSnC7D1qNUemiDbmtaMvF0saiFzyyW15erffxGCuQyiFg

In [7]:
# open the keys file
my_file = open("keys.txt", "r")

# read the raw data
content = my_file.read()

# split all lines by  newline character
API_KEY, API_SECRET_KEY = content.split("\n")

# close the file
my_file.close()

In [13]:
twitter = Twython(API_KEY, API_SECRET_KEY)

authentication_tokens = twitter.get_authentication_tokens()
print(authentication_tokens['auth_url'])

https://api.twitter.com/oauth/authenticate?oauth_token=LPL9XAAAAAABZBP-AAABftrkV8E


## 2.2 Authorized tokens 

In [14]:
# Replace the verifier with the pin number obtained with your web browser in the previous step
VERIFIER = '3396549'

twitter = Twython(API_KEY, API_SECRET_KEY,
                  authentication_tokens['oauth_token'],
                  authentication_tokens['oauth_token_secret'])

authorized_tokens = twitter.get_authorized_tokens(VERIFIER)

## 2.3 Use authorized tokens 

In [16]:
twitter = Twython(API_KEY, API_SECRET_KEY,
                  authorized_tokens['oauth_token'],
                  authorized_tokens['oauth_token_secret'])

twitter.verify_credentials()

{'id': 1486095105811484677,
 'id_str': '1486095105811484677',
 'name': 'Renato Farias',
 'screen_name': 'renatolinsfar',
 'location': '',
 'description': '',
 'url': None,
 'entities': {'description': {'urls': []}},
 'protected': False,
 'followers_count': 9,
 'friends_count': 50,
 'listed_count': 2,
 'created_at': 'Tue Jan 25 21:55:10 +0000 2022',
 'favourites_count': 0,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'verified': False,
 'statuses_count': 1,
 'lang': None,
 'status': {'created_at': 'Sat Jan 29 01:36:43 +0000 2022',
  'id': 1487238244727238656,
  'id_str': '1487238244727238656',
  'text': 'Olá, eu acabei de publicar no Medium uma matéria sobre meu trabalho de Análise de Redes usando o dataset “Spotify M… https://t.co/UsjSkojKUd',
  'truncated': True,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [],
   'urls': [{'url': 'https://t.co/UsjSkojKUd',
     'expanded_url': 'https://twitter.com/i/web/status/1487238244727238656',
     'di

# 3.0 Twitter API basics 

We've already seen one User Object, the one corresponding to the logged-in user, via the call to verify_credentials() above. We can also fetch data for an arbitrary user:

- Acredito que em nosso projeto, não usaremos essa funcionalidade. 

# 4.0 Using Twitter's search API to get tweets of interest

Twitter's Search API allows you to fetch recent tweets according to a query of keywords, URLs, hashtags, user mentions, etc.

# 4.1 Get tweets matching a hashtag 

- Essa será a principal funcionalidade do projeto. 

 - Vamos usar essa funcionalidade para procurar tweets com a hashtag que irá ser o tema para a contrução da nossa rede. 

In [17]:
search_response = twitter.search(q='#NFL', count=100)

In [18]:
search_response

{'statuses': [{'created_at': 'Tue Feb 08 21:31:32 +0000 2022',
   'id': 1491162806779285504,
   'id_str': '1491162806779285504',
   'text': 'Over 1.9 million plays!\nDownload Beats Instantly!\nhttps://t.co/mTe1ONzsr6\n\n#espn #nba #sports #nfl #football… https://t.co/kkItd4M1Xd',
   'truncated': True,
   'entities': {'hashtags': [{'text': 'espn', 'indices': [75, 80]},
     {'text': 'nba', 'indices': [81, 85]},
     {'text': 'sports', 'indices': [86, 93]},
     {'text': 'nfl', 'indices': [94, 98]},
     {'text': 'football', 'indices': [99, 108]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/mTe1ONzsr6',
      'expanded_url': 'https://air.bi/UgrOS',
      'display_url': 'air.bi/UgrOS',
      'indices': [50, 73]},
     {'url': 'https://t.co/kkItd4M1Xd',
      'expanded_url': 'https://twitter.com/i/web/status/1491162806779285504',
      'display_url': 'twitter.com/i/web/status/1…',
      'indices': [110, 133]}]},
   'metadata': {'iso_language_code': 'en',

## 4.2 Using a cursor to get more than 100 tweets from a search 

- Explorando os limites da API na captura de dados. 

In [ ]:
# even with count=1000, we still get at most 100 tweets
search_response = twitter.search(q='#NFL', count=1000)
len(search_response['statuses'])

In order to get more tweets from a search, we can make use of a cursor:

In [ ]:
import itertools

NUM_TWEETS_TO_FETCH = 1000

cursor = twitter.cursor(twitter.search, q='#NFL', count=100, result_type='mixed')
search_tweets = list(itertools.islice(cursor, NUM_TWEETS_TO_FETCH))
len(search_tweets)

In [ ]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for tweet in search_tweets:
    dict_['user'].append(tweet['user']['screen_name'])
    dict_['date'].append(tweet['created_at'])
    dict_['text'].append(tweet['text'])
    dict_['favorite_count'].append(tweet['favorite_count'])
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)

In [ ]:
df.head()

In [ ]:
df.shape

# 5.0 Creating Twitter mention network

- Aqui começa a análise dos dados coletados. 

- A rede será criada de acordo com os steets que possuem o conteúdo "NFL".

- Será análisado os usuários mais citados dentro do contexto NFL. 

## 5.1 Get some tweets

- Limit API

In [159]:
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 0, 'reset': 1644725977}}

- Para extrapolar o uso da API, iremos usar a função "sleep", para usar as 180 requisições a cada 15 minutos, extraindo assim o máximo de tweets em um curto intervalo de tempo.

In [158]:
import itertools
import time 

NUM_TWEETS_TO_FETCH = 17900

cursor = twitter.cursor(twitter.search, q='NFL', count=100, result_type='mixed')
search_tweets += list(itertools.islice(cursor, NUM_TWEETS_TO_FETCH))
len(search_tweets)

35800

In [144]:
#search_tweets.clear()

In [160]:
len(search_tweets)

35800

- Aqui os tweets estão em formato de lista. 

In [123]:
#search_tweets

- Na célula abaixo, iremos filtra apenas os itens que nos interessa, e transforma a lista em um dataframe pandas.

In [161]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'location':[], 'favorite_count': []}
for tweet in search_tweets:
    dict_['user'].append(tweet['user']['screen_name'])
    dict_['date'].append(tweet['created_at'])
    dict_['text'].append(tweet['text'])
    dict_['location'].append(tweet['user']['location'])
    dict_['favorite_count'].append(tweet['favorite_count'])
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)

In [49]:
#df.head()

,user,date,text,location,favorite_count
0,AdamSchefter,Wed Feb 09 22:20:33 +0000 2022,ESPN expanded its agreement with Peyton and El...,New York,48236
1,ChrisVannini,Wed Feb 09 16:31:52 +0000 2022,Petition to bring back fun Super Bowl logos. \...,Dallas (imported from Detroit),24427
2,JimTrotter_NFL,Wed Feb 09 21:10:48 +0000 2022,"The NFL says diversity, equity and inclusion a...","San Diego, CA",10009
3,Og_Django,Thu Feb 10 17:56:50 +0000 2022,@NFL @NFLALLDAY Jamar chase,florida,0
4,goatedshit,Thu Feb 10 17:56:49 +0000 2022,@brgridiron @NFL @NFLFilms Bills gonna punish ...,Top of the buildin,0


In [162]:
df.shape

(35800, 5)

## 5.2 Baixando o Dataset 

- Fazendo o download do dataset por dia. 

In [138]:
df.to_excel("day2.xlsx")

- Salvano dados do dia 1.

In [141]:
day2.shape 

(89500, 5)

In [101]:
day1.shape 

(89500, 5)

## 5.3 Creating DiGraph 
It's not necessary to first filter out tweets containing user mentions due to a feature of Tweet Objects: every tweet has Entities which always contains a 'user_mentions' list, even if that list is empty. Since a tweet may mention more than one user, we need a nested for-loop.

In [ ]:
search_tweets[0]

In [ ]:
import networkx as nx

D = nx.DiGraph()

for tweet in search_tweets:
    tweet_sn = tweet['user']['screen_name']
    for user_mention in tweet['entities']['user_mentions']:
        mentioned_sn = user_mention['screen_name']
        
        my_edge = (tweet_sn, mentioned_sn)
        if D.has_edge(*my_edge):
            D.edges[my_edge]['weight'] += 1
        else:
            D.add_edge(*my_edge, weight=1)